In [7]:
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")
pd.set_option('expand_frame_repr', False)

In [8]:
path = "C:/Users/chial/Downloads/data.xlsx" 
ads_table = pd.read_excel(path, sheet_name = 0, header = 1)
mod_table = pd.read_excel(path, sheet_name = 1)

In [9]:
mod_table = mod_table[~(mod_table['Productivity'].isna() & mod_table['Utilisation %'].isna() & (mod_table['handling time'] == 0))]
mod_table['accuracy'] = pd.to_numeric(mod_table['accuracy'], errors='coerce')
mod_table = mod_table.dropna(subset=['accuracy'])
mod_table = mod_table.reset_index(drop=True)

## Functions

In [16]:
# Generate score as in Problem A
# TO BE CHANGED
def fake_score():
    return random.random()

# Get list of Moderators operating in the targeted market 
def check_moderator_market(target_market):
    return mod_table[mod_table['market'].apply(lambda x: target_market in x)]

# Allocate risk score to the job
def tag_job_with_risk (job):
    if job['score']>0.75:
        return 2
    elif job['score']>0.25:
        return 1
    else: return 0

# Allocate moderator accuracy 
def tag_moderator_with_score(moderator_pool):
    high_score_cutoff = moderator_pool['accuracy'].quantile(0.75)
    mid_score_cutoff = moderator_pool['accuracy'].quantile(0.25)

    moderator_pool['Score Segment'] = np.where(moderator_pool['accuracy'] >= high_score_cutoff, 2,
                             np.where(moderator_pool['accuracy'] >= mid_score_cutoff, 1, 0))

# Shortlist moderators who are accurate enough for the review
def match_moderator_score_to_job(moderator_pool, job):
    moderator_pool.drop(moderator_pool[moderator_pool['Score Segment'] != job['Risk Segment']].index, inplace=True)

# Function to return number of jobs given ID of moderator
# TO BE CHANGED
def number_of_jobs(id):
    return 1

# Function to get number of jobs each moderator in the moderator pool has 
def number_of_jobs_for_each_moderator(moderator_pool):
    moderator_pool['Number of Jobs'] = 0
    moderator_pool['Number of Jobs'] = moderator_pool['Number of Jobs'].apply(lambda x: number_of_jobs(moderator_pool['moderator']))

def least_job_policy(moderator_pool):
    least_jobs_value = moderator_pool['Number of Jobs'].min()
    moderator = moderator_pool.loc[moderator_pool['Number of Jobs'] == least_jobs_value, 'moderator'].values[0]
    return moderator

def least_job_hour_policy(moderator_pool):
    moderator_pool['Number of Job Hours'] = 0
    moderator_pool['Number of Job Hours'] = moderator_pool['Number of Jobs']*moderator_pool['handling time']
    least_job_hour_value = moderator_pool['Number of Job Hours'].min()
    moderator = moderator_pool.loc[moderator_pool['Number of Job Hours'] == least_job_hour_value, 'moderator'].values[0]
    print(moderator_pool)
    return moderator

def assign_job_to_moderator(moderator_pool, job):
    job['Risk Segment'] = tag_job_with_risk(job)
    moderator_pool = check_moderator_market(job['delivery_country'])
    tag_moderator_with_score(moderator_pool)
    match_moderator_score_to_job(moderator_pool, job)
    number_of_jobs_for_each_moderator(moderator_pool)
    # return least_job_policy(moderator_pool)
    return least_job_hour_policy(moderator_pool)

In [17]:
# Get a job as a dataframe of 1 row 
ads_table['score'] = 0
ads_table['score'] = ads_table['score'].apply(lambda x: fake_score())
job = ads_table.iloc[0]

In [18]:
moderator_pool = mod_table
assign_job_to_moderator(moderator_pool, job)

             moderator        market  Productivity  Utilisation %  handling time  accuracy  Total time reviewing  Total work time  New productivity  New Total Time Reviewing  Total work time.1  New Utilisation  Score Segment  Number of Jobs  Number of Job Hours
50    1672502123346950  ["US", "CA"]    133.226667       0.537611          79835     0.789          1.063615e+07     1.978410e+07               NaN                         0       1.978410e+07              0.0              1               1                79835
53    1752289495219201  ["US", "CA"]    383.448000       0.805644          97686     0.911          3.745750e+07     4.649384e+07               NaN                         0       4.649384e+07              0.0              1               1                97686
64    1759961674413106        ["US"]    261.977500       0.808214          89468     0.792          2.343860e+07     2.900051e+07               NaN                         0       2.900051e+07              0.0     

1598527